In [1]:
import sys
import os
from typing import Tuple
import pandas as pd 
import mlflow as mf

## replace with your specific paths
sys.path.append("/home/aamir07/mlops2_with_dagster/") 
ARTIFACT_PATH = "/home/aamir07/mlops2_with_dagster/artifacts/mlflow_artfacts" 
LOGGER_FOLDER_PATH = "/home/aamir07/mlops2_with_dagster/notebooks"

from mlops2_with_dagster import encoder_pipeline
from pathlib import Path
from joblib import dump, load
from hamilton import driver, base
from mlops2_with_dagster.utils import get_project_dir, printse
from sklearn.preprocessing import (
    StandardScaler,
    LabelEncoder
)

from ortho import BaseKernel
from ortho.utils import Logger
from ortho.ortho.decorators import task
from ortho.ortho.callbacks import MlFlowCallBack, LoggerCallBack


logger = Logger().logger()

   

        
        

None


2023-09-15 00:04:24.088777: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:04:24.198567: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-15 00:04:24.201212: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 00:04:26,844 - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-09-15 00:04:27.029518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-15 00:04:27,112 - DEBUG - Creating converter from 7 to 5
2023-09-15 00:04:27,115 - DEBUG - Creating converter from 5 to 7
2

In [2]:
project = 'mlops2_with_dagster'
project_dir = get_project_dir(project)
printse(f'project_dir: {project_dir}')

>>>> project_dir: /home/aamir07/mlops2_with_dagster


In [3]:
class TargetExtractor(BaseKernel):
    """
    This class is responsible for extracting the target variable from the the the training dataset
    """
    
    def __init__(self, callbacks,  experiment_name, run_name, load_from_artifact=False):
        super().__init__(callbacks,
                       experiment_name,
                       run_name,
                       load_from_artifact)
        
        self.index_col = 'passengerid'
        self.target_col = "survived"
        self.cat_cols = ["sex", "cabin", "embarked"]
        self.config = {
            'index_column': self.index_col,
            'target_column': self.target_col,
            'categorical_columns': self.cat_cols
        }
    
    @task(build_on_previous_run=False, end_mlflow_run=True)    # This mean that this is an independent run and might have child runs in future  
    def get_target(self, df_train):
        encode_dr = driver.Driver(self.config, encoder_pipeline)
        output_nodes = ['target']
        
        out = encode_dr.execute(output_nodes ,
        inputs = dict(
            df_train = df_train
        )         
        )
        
        data = {"target": out }
        payload = {"artifact_path": ARTIFACT_PATH}
        
        return data, payload
    
    def run(self, *args, **kwargs):
        
        return self.get_target(*args, **kwargs)
        
        
    

In [4]:
train_data : Path = project_dir/"data/train.csv"
test_data : Path = project_dir/"data/test.csv"

In [5]:
# parameters
df_train = pd.read_csv(train_data)

In [6]:
import dagstermill
extractor = TargetExtractor(experiment_name="Mlflow_with_Dagster",
                          run_name="BadamBhum",
                          callbacks = [LoggerCallBack(log_folder=LOGGER_FOLDER_PATH, 
                                                      kernel_names=["get_target"]),
                                       MlFlowCallBack(kernel_names=["get_target"])])

# If you run into an error and process finishes prematurely use mf.end_run() to end the active mlflow run
dagstermill.yield_result(extractor.run(df_train=df_train)[0]["target"], output_name="target")

2023-09-15 00:04:29,288 - DEBUG - Starting new HTTP connection (1): 127.0.0.1:5002


File not found: /home/aamir07/dagster_pipeline_2/dagster_pipeline_2/env/dev/.env


2023-09-15 00:04:30,323 - DEBUG - http://127.0.0.1:5002 "GET /api/2.0/mlflow/experiments/get-by-name?experiment_name=Mlflow_with_Dagster HTTP/1.1" 200 241
2023-09-15 00:04:30,648 - DEBUG - Popen(['git', 'version'], cwd=/home/aamir07/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-09-15 00:04:30,653 - DEBUG - Popen(['git', 'version'], cwd=/home/aamir07/mlops2_with_dagster/notebooks, universal_newlines=False, shell=None, istream=None)
2023-09-15 00:04:30,739 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:04:30,967 - DEBUG - http://127.0.0.1:5002 "POST /api/2.0/mlflow/runs/create HTTP/1.1" 200 894
2023-09-15 00:04:30,970 - INFO - Successfuly initiated run with name BadamBhum and id b597418f95f441e3862419c83722087a
2023-09-15 00:04:30,974 - WARNING - Note: Hamilton collects completely anonymous data about usage. This will help us improve Hamilton over time. See https://github.com/dagworks-inc/hamilton#usage-analytics--data-privacy fo

Error saving environment variables: [Errno 2] No such file or directory: '/home/aamir07/dagster_pipeline_2/dagster_pipeline_2/env/dev/.env'


2023-09-15 00:04:31,305 - DEBUG - http://127.0.0.1:5002 "PUT /api/2.0/mlflow-artifacts/artifacts/1/b597418f95f441e3862419c83722087a/artifacts/target.pickle HTTP/1.1" 200 2
2023-09-15 00:04:31,309 - DEBUG - Resetting dropped connection: 127.0.0.1
2023-09-15 00:04:31,340 - DEBUG - Succeed in sending telemetry consisting of [b'{"api_key": "phc_mZg8bkn3yvMxqvZKRlMlxjekFU5DFDdcdAsijJ2EH5e", "event": "os_hamilton_run_start", "properties": {"os_type": "posix", "os_version": "Linux-5.15.90.1-microsoft-standard-WSL2-x86_64-with-glibc2.35", "python_version": "3.10.11/CPython", "distinct_id": "f41b7308-274c-4333-ab85-eca6fe911802", "hamilton_version": [1, 28, 0], "telemetry_version": "0.0.1", "number_of_nodes": 32, "number_of_modules": 1, "number_of_config_items": 3, "decorators_used": {"parameterize_sources": 4, "extract_columns": 2, "extract_fields": 1}, "graph_adapter_used": "hamilton.base.SimplePythonDataFrameGraphAdapter", "result_builder_used": "hamilton.base.PandasDataFrameResult", "driver

,target
passengerid,
0,1
1,0
2,0
3,0
4,1
...,...
99995,1
99996,0
99997,0
